<a href="https://colab.research.google.com/github/Anson422/Anson-Eva-Valerie/blob/main/Choropleth%20Map%20Analysis/choropleth_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geojson

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import plotly
import plotly.express as px
import warnings
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
sns.set(font_scale=1.5)
sns.set_style("white")
plt.rcParams['figure.figsize'] = (12, 8)

In [8]:
import geojson
import json
import plotly.graph_objects as go
from google.colab import files
uploaded = files.upload()


Saving Natural gas dependency.xlsx to Natural gas dependency (1).xlsx


In [35]:
df = pd.read_excel("Natural gas dependency.xlsx")
df.head()

,Country,2015,2016,2017,2018,2019,2020
0,Belgium,3.69,7.93,6.81,11.45,16.42,14.45
1,Bulgaria,100,100,100,99.92,85.73,83.77
2,Czechia,99.81,99.97,99.19,99.5,99.73,100
3,Denmark,:,:,:,:,:,62.32
4,Germany,45.32,60.25,54.34,51.22,51.22,68.66


In [36]:

# Melt the DataFrame into panel data format
df_melt = pd.melt(df, id_vars=['Country'], var_name='year', value_name='dependency(%)')

# Ensure correct data types
df_melt['Country'] = df_melt['Country'].astype(str)
df_melt['year'] = pd.to_numeric(df_melt['year'])
df_melt['dependency(%)'] = pd.to_numeric(df_melt['dependency(%)'], errors='coerce')

# Handle missing values (choose your preferred method)
df_melt = df_melt.dropna()  # Example: drop rows with missing values

# Sort the data
df_melt = df_melt.sort_values(['Country', 'year'])
df_melt


,Country,year,dependency(%)
0,Belgium,2015,3.69
40,Belgium,2016,7.93
80,Belgium,2017,6.81
120,Belgium,2018,11.45
160,Belgium,2019,16.42
...,...,...,...
29,United Kingdom,2015,0.61
69,United Kingdom,2016,1.15
109,United Kingdom,2017,0.66
149,United Kingdom,2018,4.50


In [6]:
uploaded = files.upload()


Saving custom.geo.json to custom.geo.json


In [30]:
europe_geojson = json.load(open("custom.geo.json"))
europe_geojson


{'type': 'FeatureCollection', 'features': []}

In [28]:
if 'features' in europe_geojson and europe_geojson['features']:
    print(europe_geojson.keys())
    features = europe_geojson['features']
    print(features[0])
else:
    print("Error: 'features' key not found or empty in GeoJSON data.")

Error: 'features' key not found or empty in GeoJSON data.


In [40]:

px.choropleth( # plot a choropleth map using the plotly express (px) library
                df_melt, # load the dataframe
                locations='Country', # set the locations to 'europe'
                locationmode='country names',
                geojson=europe_geojson,
                scope='europe',
                featureidkey="properties.name",
                color="dependency(%)", # set the color of the states to correspond to the unemployment rate
                animation_frame=df_melt["year"].astype(str), # set the animation frame to the date, creating a slider at the bottom of the map
                color_continuous_scale=px.colors.sequential.Viridis[::-1], # set the color scale to Viridis, a commonly used color scale
                range_color=[0, 100], # set the range of the color scale to 0-100
                height=1000) # set the height of the map to 1000 pixels